# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,7.87,87,77,1.34,US,1701707711
1,1,port elizabeth,-33.9180,25.5701,20.17,76,20,8.75,ZA,1701707828
2,2,codrington,-38.2667,141.9667,12.03,94,86,0.39,AU,1701707829
3,3,jamestown,42.0970,-79.2353,4.77,88,100,8.75,US,1701707829
4,4,grytviken,-54.2811,-36.5092,1.80,63,41,6.47,GS,1701707829


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", 
                                        geo = True,
                                        color = "City",
                                        alpha = .5,
                                        size = "Humidity",
                                        tiles = "OSM")

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 21)
                               & (city_data_df["Max Temp"] > 16)
                               & (city_data_df["Cloudiness"] < 2)
                               & (city_data_df["Wind Speed"] < 2)
                               ]
# Drop any rows with null values
filtered_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,sitia,35.2000,26.0986,18.32,69,0,0.61,GR,1701707700
116,116,heroica guaymas,27.9333,-110.9000,20.49,41,0,0.00,MX,1701707834
129,129,bhadasar,28.2833,74.3167,17.60,34,0,1.00,IN,1701707858
182,182,greenville,35.6127,-77.3663,18.01,46,0,1.54,US,1701707700
198,198,alice springs,-23.7000,133.8833,20.75,64,0,0.00,AU,1701707871
399,399,guerrero negro,27.9769,-114.0611,20.37,40,0,0.95,MX,1701707843
482,482,fort bragg,35.1390,-79.0060,19.34,45,0,0.89,US,1701707870


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/mq/fw12kwg17c10hbtsjkrxkm4w0000gn/T/ipykernel_73992/3139780859.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
30,sitia,GR,35.2000,26.0986,69,
116,heroica guaymas,MX,27.9333,-110.9000,41,
129,bhadasar,IN,28.2833,74.3167,34,
182,greenville,US,35.6127,-77.3663,46,
198,alice springs,AU,-23.7000,133.8833,64,
399,guerrero negro,MX,27.9769,-114.0611,40,
482,fort bragg,US,35.1390,-79.0060,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accomodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity: {longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sitia - nearest hotel: No hotel found
heroica guaymas - nearest hotel: No hotel found
bhadasar - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,sitia,GR,35.2000,26.0986,69,No hotel found
116,heroica guaymas,MX,27.9333,-110.9000,41,No hotel found
129,bhadasar,IN,28.2833,74.3167,34,No hotel found
182,greenville,US,35.6127,-77.3663,46,No hotel found
198,alice springs,AU,-23.7000,133.8833,64,No hotel found
399,guerrero negro,MX,27.9769,-114.0611,40,No hotel found
482,fort bragg,US,35.1390,-79.0060,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", 
                                        geo = True,
                                        color = "City",
                                        alpha = .5,
                                        size = "Humidity",
                                        tiles = "OSM",
                                        hover_cols = ["Hotel Name", "Country"]
                                        )
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)